In [10]:
import numpy as np
import pandas as pd

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.base import backtest_df_only, order_records_bt
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.strategies.strategy_example import StrategyExample

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
candles = mufex.get_candles_df(
    symbol=symbol, timeframe='5m', candles_to_dl=300)

It took 00 mins and 01 seconds to download 300 candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    long_or_short=np.array([LongOrShortType.Long]),
    increase_position_type=np.array(
        [IncreasePositionType.RiskPctAccountEntrySize]),
    risk_account_pct_size=np.array([1]),
    max_equity_risk_pct=np.array([3]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]),
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]),
    trail_sl_by_pct=np.array([1.0]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    risk_reward=np.array([3, 5]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    static_leverage=np.array([0.0]),
    num_candles=np.array([200]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),

)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [4]:
strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14],
    rsi_is_below=[50, 80],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
    candles=candles,
)


Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 2
Total order settings to test: 2
Total combinations of settings to test: 4

Total candles: 300
Total candles to test: 1,200


In [5]:
candles_reg.sort_values(['ind_set_idx', 'or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
1,0,0,23.0,-18.95,13.04,-0.0000,-189.4990,810.5010
0,0,1,21.0,-18.49,14.29,-0.0000,-184.8554,815.1446
3,1,0,22.0,-19.38,13.64,-0.8268,-193.8011,806.1989
2,1,1,21.0,-20.67,14.29,-0.0000,-206.7248,793.2752


In [6]:
candles_reg.iloc[3][['ind_set_idx', 'or_set_idx']].astype(int).values

array([1, 0])

In [7]:
order_records = order_records_bt(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
    candles=candles,
    backtest_results=candles_reg,
    backtest_index=1,
)
order_records.timestamp = pd.to_datetime(order_records.timestamp, unit="ms")

In [15]:
for i in range(len(OrderStatus._fields)):
    order_records.replace(
        {"order_status": {i: OrderStatus._fields[i]}}, inplace=True
    )

In [88]:
timestamps_2d_list = np.reshape(candles['timestamp'], (-1, 1)).tolist()

In [52]:
temp_df = order_records[order_records["order_status"] == "EntryFilled"][
    ["bar_idx", "entry_price", "average_entry", "liq_price", "sl_price", "tp_price"]
]

In [54]:
temp_df.head(4)

,bar_idx,entry_price,average_entry,liq_price,sl_price,tp_price
0,26,28030.1,28030.1,27689.6,27717.3,29153.0
1,30,28044.3,28036.0,27846.7,27874.6,28717.2
2,32,28054.7,28041.7,27884.7,27912.6,28621.3
4,77,28385.5,28385.5,28134.0,28162.2,29242.4


In [102]:
timestamps_2d_list = np.reshape(candles['timestamp'], (-1, 1)).tolist()
entry_list = timestamps_2d_list[:].copy()
avg_entry_list = entry_list[:].copy()
liq_price_list = timestamps_2d_list[:]
sl_price_list = timestamps_2d_list[:]
tp_price_list = timestamps_2d_list[:]

In [103]:
for _, value in temp_df.T.items():
    bar_index = int(value['bar_idx'])
    entry_list[bar_index].append(value['entry_price'])
    avg_entry_list[bar_index].append(value['average_entry'])
    liq_price_list[bar_index].append(value['liq_price'])
    sl_price_list[bar_index].append(value['sl_price'])
    tp_price_list[bar_index].append(value['tp_price'])

In [104]:
liq_price_list

[[1697462100000],
 [1697462400000],
 [1697462700000],
 [1697463000000],
 [1697463300000],
 [1697463600000],
 [1697463900000],
 [1697464200000],
 [1697464500000],
 [1697464800000],
 [1697465100000],
 [1697465400000],
 [1697465700000],
 [1697466000000],
 [1697466300000],
 [1697466600000],
 [1697466900000],
 [1697467200000],
 [1697467500000],
 [1697467800000],
 [1697468100000],
 [1697468400000],
 [1697468700000],
 [1697469000000],
 [1697469300000],
 [1697469600000],
 [1697469900000, 28030.1, 28030.1, 27689.6, 27717.3, 29153.0],
 [1697470200000],
 [1697470500000],
 [1697470800000],
 [1697471100000, 28044.3, 28036.0, 27846.7, 27874.6, 28717.2],
 [1697471400000],
 [1697471700000, 28054.7, 28041.7, 27884.7, 27912.6, 28621.3],
 [1697472000000],
 [1697472300000],
 [1697472600000],
 [1697472900000],
 [1697473200000],
 [1697473500000],
 [1697473800000],
 [1697474100000],
 [1697474400000],
 [1697474700000],
 [1697475000000],
 [1697475300000],
 [1697475600000],
 [1697475900000],
 [1697476200000],
 